In [78]:
#!pip install --upgrade tensorflow-datasets

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from bs4 import BeautifulSoup 

import re
from nltk.corpus import stopwords 

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Data Loading

In [62]:
ds = tfds.load('reddit_tifu')
df = tfds.as_dataframe(ds['train'].take(30000))

#We only need these two columns for summarization
df = df[['documents','title']]

# Convert object to string
df = df.astype(str)

In [63]:
#Remove the first two characters
df['documents'] = df.documents.str.slice_replace(stop=2, repl ='')
df['title'] = df.title.str.slice_replace(stop=2, repl ='')

# Preprocessing

In [53]:
#Contraction mapping
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [57]:
#get stopwords from NLTK
stop_words = set(stopwords.words('english')) 

#this function cleans the text
def text_cleaner(text):
    """
    This function cleanes the texts and returns the cleaned senctance.
    
    Input:
    text - Any string
    
    Output:
    Sentance - Cleaned string
    """
    
    newString = text.lower()
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = [w for w in newString.split() if not w in stop_words]
    long_words=[]
    
    for i in tokens:
        if len(i)>=3:                  #removing short word
            long_words.append(i)   
            
    return (" ".join(long_words)).strip()

In [64]:
#Applying the functions to clean each column
df.documents = df.documents.apply(text_cleaner)
df.title = df.title.apply(text_cleaner)

In [103]:
df.title = df.title.apply(lambda x : '_START_ '+ x + ' _END_')

# Tokenizing

In [105]:
# Maximum length for each input and target 
max_text_len = 120
max_summary_len = 15

In [106]:
#Train and test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.documents, df.title, test_size = 0.1, shuffle = True)

In [107]:
def sequence_tokenizer(train, test, max_len):
    """
    This function tokenizes and adds padding to a sequence of texts.
    
    Input:
    train - Train data which contains sentances.
    test - Test data, which also contains sentances.
    max_len - Argument to specify the maximum length for padding.
    
    Output:
    train - Toeknized train data
    test - Tokenized test data
    tokenizer - The tokenizer instance ( To invert back in the future )
    """
    
    #Initialize the tokenizer
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train)
    
    #Texts to integer squences
    train = tokenizer.texts_to_sequences(train)
    test = tokenizer.texts_to_sequences(test)
    
    #Add padding
    train = pad_sequences(train, maxlen= max_len, padding = 'post')
    test = pad_sequences(test, maxlen= max_len, padding = 'post')
    
    return train, test, tokenizer
    

In [108]:
X_train, X_test, X_tokenizer = sequence_tokenizer(X_train, X_test, max_text_len)
y_train, y_test, y_tokenizer = sequence_tokenizer(y_train, y_test, max_summary_len)